# Compare the composition files

Define the excel_file name

In [1]:
from pathlib import Path

compositions = [
    "20250819__Cacao__AltaVerapaz",
    "20250819__Cacao__DR_Cibao_Noroeste",
    "20250819__Coffee__DR_Cibao_Noroeste",
    "20250819__Coffee__Veracruz",
    "20250819__Coffee__WesternHighlands"
]

prefix = compositions[0]  # Extract the crop name from the first composition

# Define output and input directories
INPUT_DIR = Path("/Users/viggo/Documents/Programming/Consult/UNU/biofincas_climate_risk/agroforestry_systems")

# Define excel_file names
file_name_Zelie = prefix + ".xlsx"
excel_file = INPUT_DIR / file_name_Zelie

Define the output directioty

In [2]:

from tabulate import tabulate

# Define output and input directories
#INPUT_DIR_VICTOR = Path("/Users/viggo/Documents/Programming/Consult/UNU/data")
OUTPUT_DIR = Path("/Users/viggo/Documents/Programming/Consult/UNU/biofincas_climate_risk/Results")
# Make a directory for the modified agroforestry systems in prefix
OUTPUT_DIR = OUTPUT_DIR / prefix
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)


#file_name_Victor = "canopy_crop_composition.xlsx"


Read Victors excel_file

In [3]:
# import pandas as pd
# from tabulate import tabulate

# # get the number of sheets in the Excel excel_file apart from the 'Current' sheet
# excel_file = INPUT_DIR_VICTOR / file_name_Victor
# canopy_comps = pd.ExcelFile(excel_file).sheet_names

# # Create a dictionary to hold the canopy composition data
# canopy_crop_victor_dict = {}
# for sheet in canopy_comps:
#     canopy_crop_victor_dict[sheet] = pd.read_excel(excel_file, sheet_name=sheet)
#     print(f"Loaded sheet: {sheet} with {len(canopy_crop_victor_dict[sheet])} rows.")
#     print(tabulate(canopy_crop_victor_dict[sheet].head(), headers='keys', tablefmt='psql'))

Zélies files

In [4]:
import pandas as pd

# get the number of sheets in the Excel excel_file apart from the 'Current' sheet
canopy_comps_zelie = pd.ExcelFile(excel_file).sheet_names

# Modify the DataFrame to get the site_id and rename columns
def modify_canopy_crop_df(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    
    # Generate unique site_id
    df["Plot ID"] = df.groupby(["Latitude", "Longitude"]).ngroup()

    # Chnage ti string
    df["Plot ID"] = df["Plot ID"].astype(str)

    return df

# Create a dictionary to hold the canopy composition data
canopy_crop_zelie_dict = {}
for sheet in canopy_comps_zelie:
    canopy_crop_zelie_dict[sheet] = pd.read_excel(excel_file, sheet_name=sheet)
    canopy_crop_zelie_dict[sheet] = modify_canopy_crop_df(canopy_crop_zelie_dict[sheet])
    print(f"Loaded sheet: {sheet} with {len(canopy_crop_zelie_dict[sheet])} rows.")
    print(tabulate(canopy_crop_zelie_dict[sheet], headers='keys', tablefmt='psql'))

Loaded sheet: present with 100 rows.
+----+--------------------+----------+------------------+-------------------+-----------------------+-------------+---------------------+------------------------+------------+-------------+-----------------+-----------+
|    | Region             | System   |   Plot size (ha) | Species           | Scientific name       |   Plants/ha |   Yield (t/ha/year) |   Per-tree shading (%) |   Latitude |   Longitude |   Elevation (m) |   Plot ID |
|----+--------------------+----------+------------------+-------------------+-----------------------+-------------+---------------------+------------------------+------------+-------------+-----------------+-----------|
|  0 | GTM - Alta Verapaz | Cacao    |                1 | Cacao (main crop) | Theobroma cacao       |         nan |                 0.5 |                    nan |    15.7075 |    -90.2486 |             366 |         3 |
|  1 | GTM - Alta Verapaz | Cacao    |                1 | Gliricidia        | Gliri

## Compare the two 'present' composition

In [5]:
# # Print only the first item
# print(f"Zélies files: \n{tabulate(canopy_crop_zelie_dict['present'], headers='keys', tablefmt='psql')}")
# print(f"Victors files:\n{tabulate(canopy_crop_victor_dict['present'], headers='keys', tablefmt='psql')}")

# # Print the columns missing in Zélies
# zelie_columns = set(canopy_crop_zelie_dict['present'].columns)
# victor_columns = set(canopy_crop_victor_dict['present'].columns)

# missing_in_zelie = victor_columns - zelie_columns
# missing_in_victor = zelie_columns - victor_columns

# print(f"Columns missing in Zélies: {missing_in_zelie}")
# print(f"Columns missing in Victor: {missing_in_victor}")

# Get all the unique Species and Species Names acrross the sheets

In [6]:
for sheet in canopy_crop_zelie_dict:
    print(f"Sheet: {sheet}")
    print(tabulate(canopy_crop_zelie_dict[sheet].head(), headers='keys', tablefmt='psql'))

Sheet: present
+----+--------------------+----------+------------------+-------------------+-----------------------+-------------+---------------------+------------------------+------------+-------------+-----------------+-----------+
|    | Region             | System   |   Plot size (ha) | Species           | Scientific name       |   Plants/ha |   Yield (t/ha/year) |   Per-tree shading (%) |   Latitude |   Longitude |   Elevation (m) |   Plot ID |
|----+--------------------+----------+------------------+-------------------+-----------------------+-------------+---------------------+------------------------+------------+-------------+-----------------+-----------|
|  0 | GTM - Alta Verapaz | Cacao    |                1 | Cacao (main crop) | Theobroma cacao       |         nan |                 0.5 |                    nan |    15.7075 |    -90.2486 |             366 |         3 |
|  1 | GTM - Alta Verapaz | Cacao    |                1 | Gliricidia        | Gliricidia sepium     |    

In [7]:
import copy
import pandas as pd

# Copy the Zélie present DataFrame to adjust it
canopy_crop_zelie_dict_adjusted = copy.deepcopy(canopy_crop_zelie_dict)

Generate unique Site ID

In [8]:
 # Generate unique site_id
for sheet, df in canopy_crop_zelie_dict_adjusted.items():
    df["site_id"] = df.groupby(["Latitude", "Longitude"]).ngroup()

Define the default values

In [9]:
# Set estimates (USD per tonne)
DR_TYPICAL_PRICE_TONNE_USD = {
    "Coffee": 2580,  # 20% premium for specialty coffee
    "Cacao":  3760,   # ≈ matches coffee’s $/ha at 0.5 t/ha
    "Banana": 330,
}

DR_TYPICAL_YIELD = { # tonnes per hectare per year
    "Coffee": 0.73,
    "Cacao":  0.50,
    "Banana": 22.64,
}

DR_TYPICAL_PLANTS_PER_HA = { # plants per hectare
    "Coffee": 3000,
    "Cacao":  1000,
    "Banana": 2000,
}

## Add Plants/ha for teh main crops coffe, cacao and banana to each sheet

In [10]:
import pandas as pd
import numpy as np


# species we care about Scientific names, species names
TARGETS = {
    "Coffea arabica":   ("Coffee (main crop)", "Coffee"),
    "Theobroma cacao":  ("Cacao (main crop)",  "Cacao"),
    "Musa spp.":        ("Banana",             "Banana"),
}

def ensure_main_species_and_update_plants(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()

    # ensure numeric for checks
    out["Yield (t/ha/year)"] = pd.to_numeric(out["Yield (t/ha/year)"], errors="coerce")
    out["Plants/ha"] = pd.to_numeric(out["Plants/ha"], errors="coerce")

    rows_to_add = []

    # group by Plot ID + Region as requested
    for (plot_id, region), g in out.groupby(["Plot ID", "Region"], dropna=False):
        base = g.iloc[0].to_dict()  # copy plot metadata

        for sci_name, (common_label, key) in TARGETS.items():
            present = g[g["Scientific name"] == sci_name]

            if not present.empty:
                # update Plants/ha only where yield is known
                idx = present.index[present["Yield (t/ha/year)"].notna()]
                if len(idx):
                    out.loc[idx, "Plants/ha"] = DR_TYPICAL_PLANTS_PER_HA[key]
            else:
                # add a new row for this plot with Plants/ha = 0
                new_row = {col: base.get(col, np.nan) for col in out.columns}
                new_row["Scientific name"] = sci_name
                if "Species" in new_row:
                    new_row["Species"] = common_label
                if "Plants/ha" in new_row:
                    new_row["Plants/ha"] = 0
                if "Yield (t/ha/year)" in new_row:
                    new_row["Yield (t/ha/year)"] = np.nan
                rows_to_add.append(new_row)

    if rows_to_add:
        out = pd.concat([out, pd.DataFrame(rows_to_add)], ignore_index=True)

    # Sort the DataFrame by Plot ID and Region
    out.sort_values(by=["Plot ID", "Region", "Scientific name"], inplace=True)

    return out

# Do it for all sheets in the Zélie dictionary
for sheet, df in canopy_crop_zelie_dict_adjusted.items():
    canopy_crop_zelie_dict_adjusted[sheet] = ensure_main_species_and_update_plants(df)
    # Print the adjusted DataFrame for the sheet
    print(f"Adjusted Zélie DataFrame for sheet '{sheet}':\n{tabulate(canopy_crop_zelie_dict_adjusted[sheet], headers='keys', tablefmt='psql')}")
    print(f"Adjusted Zélie DataFrame for sheet '{sheet}' has {len(canopy_crop_zelie_dict_adjusted[sheet])} rows.")


Adjusted Zélie DataFrame for sheet 'present':
+-----+--------------------+----------+------------------+--------------------+-----------------------+-------------+---------------------+------------------------+------------+-------------+-----------------+-----------+-----------+
|     | Region             | System   |   Plot size (ha) | Species            | Scientific name       |   Plants/ha |   Yield (t/ha/year) |   Per-tree shading (%) |   Latitude |   Longitude |   Elevation (m) |   Plot ID |   site_id |
|-----+--------------------+----------+------------------+--------------------+-----------------------+-------------+---------------------+------------------------+------------+-------------+-----------------+-----------+-----------|
|  73 | GTM - Alta Verapaz | Cacao    |                1 | Cedar              | Cedrela odorata       |          15 |               nan   |                     50 |    15.5566 |    -90.0506 |             716 |         0 |         0 |
| 100 | GTM - Alta

## Get the unique species, scientific names and shade levels across all alternatives 

In [11]:


# For all sheets except 'present', only store the columns
unique_cols = ["site_id", "Species", "Scientific name", "Per-tree shading (%)"]

# 1) Stack all sheets, tagging each row with its sheet idx
stacked = pd.concat(
    [df[unique_cols].assign(Source=idx)
     for idx, (_, df) in enumerate(canopy_crop_zelie_dict_adjusted.items())],
    ignore_index=True
)

# 2) Prefer rows that HAVE shade; within those, prefer the smallest sheet idx
stacked["_shade_missing"] = stacked["Per-tree shading (%)"].isna()
stacked = stacked.sort_values(
    ["site_id", "Species", "Scientific name", "_shade_missing", "Source"],
    ascending=[True, True, True, True, True]   # non-NaN first (False < True), then lowest idx
)

# 3) Keep the first occurrence per (site_id, Species, Scientific name)
df_unique_species = (
    stacked
      .drop_duplicates(subset=["site_id", "Species", "Scientific name"], keep="first")
      .drop(columns=["_shade_missing"])
      .reset_index(drop=True)
)[["site_id", "Species", "Scientific name", "Per-tree shading (%)", "Source"]]

# If you want to see it:
# from tabulate import tabulate
print(tabulate(df_unique_species, headers='keys', tablefmt='psql'))

# Drop the 'Source' column if not needed
df_unique_species = df_unique_species.drop(columns=["Source"])
print(f"Unique species DataFrame has {len(df_unique_species)} rows.")

+-----+-----------+--------------------+-----------------------+------------------------+----------+
|     |   site_id | Species            | Scientific name       |   Per-tree shading (%) |   Source |
|-----+-----------+--------------------+-----------------------+------------------------+----------|
|   0 |         0 | Avocado            | Persea americana      |                   50   |        0 |
|   1 |         0 | Banana             | Musa paradisiaca      |                   25   |       15 |
|   2 |         0 | Banana             | Musa spp.             |                  nan   |        0 |
|   3 |         0 | Cacao (main crop)  | Theobroma cacao       |                  nan   |        0 |
|   4 |         0 | Cedar              | Cedrela odorata       |                   50   |        0 |
|   5 |         0 | Citrus             | Citrus sinensis       |                   35   |       15 |
|   6 |         0 | Coffee (main crop) | Coffea arabica        |                  nan   |  

Adjust so the other sheets only have limited columns

In [12]:
import copy

# For all sheets except 'present', only store the columns 
store_columns = ["Plot ID", "Latitude", "Longitude", "Region", "System", "Plot size (ha)", "Species", "Scientific name", "Plants/ha"]
for sheet in canopy_crop_zelie_dict_adjusted:
    if sheet != "present":
        canopy_crop_zelie_dict_adjusted[sheet] = canopy_crop_zelie_dict_adjusted[sheet][store_columns]

# For each sheet in the Zélies data, add the region key
for sheet in canopy_crop_zelie_dict_adjusted:
    canopy_crop_zelie_dict_adjusted[sheet]["Region_key"] = canopy_crop_zelie_dict_adjusted[sheet]["Region"].str.split(" - ", n=1).str[0].str.strip()


In [13]:
for sheet, df in canopy_crop_zelie_dict_adjusted.items():
    print(f"Sheet: {sheet}")
    print(tabulate(df, headers='keys', tablefmt='psql'))

Sheet: present
+-----+--------------------+----------+------------------+--------------------+-----------------------+-------------+---------------------+------------------------+------------+-------------+-----------------+-----------+-----------+--------------+
|     | Region             | System   |   Plot size (ha) | Species            | Scientific name       |   Plants/ha |   Yield (t/ha/year) |   Per-tree shading (%) |   Latitude |   Longitude |   Elevation (m) |   Plot ID |   site_id | Region_key   |
|-----+--------------------+----------+------------------+--------------------+-----------------------+-------------+---------------------+------------------------+------------+-------------+-----------------+-----------+-----------+--------------|
|  73 | GTM - Alta Verapaz | Cacao    |                1 | Cedar              | Cedrela odorata       |          15 |               nan   |                     50 |    15.5566 |    -90.0506 |             716 |         0 |         0 | GTM 

Adjust the 'present' sheet so to have a specific Species                   | Scientific name    |   Per-tree shading (%) taken from the other sheets 

In [14]:
import pandas as pd
import numpy as np

import pandas as pd
import numpy as np

def backfill_species_any_sheet(
    sheet_df: pd.DataFrame,
    df_unique_species: pd.DataFrame,
    present_df_for_site_map: pd.DataFrame
) -> pd.DataFrame:
    """
    Ensure each site (by site_id) in `sheet_df` contains all (Species, Scientific name)
    pairs from `df_unique_species` for that site. Append missing rows with Plants/ha = 0.
    If 'Per-tree shading (%)' exists in sheet_df, copy it from df_unique_species.

    site_id mapping:
      1) by exact 'Plot ID' to present
      2) if still missing, by numeric suffix of Plot ID (e.g., 001, 002, 003)
    """
    out = sheet_df.copy()

    needed_min = {"Plot ID", "Region", "System", "Species", "Scientific name", "Plants/ha"}
    missing = needed_min - set(out.columns)
    if missing:
        raise ValueError(f"Sheet is missing required columns: {missing}")

    out["Plants/ha"] = pd.to_numeric(out["Plants/ha"], errors="coerce")

    need_unique = {"site_id","Species","Scientific name","Per-tree shading (%)"}
    if not need_unique.issubset(df_unique_species.columns):
        raise ValueError("df_unique_species must have: site_id, Species, Scientific name, Per-tree shading (%)")

    # --- build site_id map from present ---
    pres = present_df_for_site_map[["Plot ID","site_id"]].drop_duplicates().copy()
    pres["_suffix"] = pres["Plot ID"].str.extract(r"(\d+)$", expand=False)

    added_temp_site = False
    if "site_id" not in out.columns:
        out = out.merge(pres[["Plot ID","site_id"]], on="Plot ID", how="left")
        added_temp_site = True

    # if still missing site_id, map by numeric suffix
    if out["site_id"].isna().any():
        out["_suffix"] = out["Plot ID"].str.extract(r"(\d+)$", expand=False)
        out = out.merge(
            pres[["_suffix","site_id"]].rename(columns={"site_id":"site_id_by_suffix"}),
            on="_suffix",
            how="left"
        )
        out["site_id"] = out["site_id"].fillna(out["site_id_by_suffix"])
        out = out.drop(columns=[c for c in ["_suffix","site_id_by_suffix"] if c in out.columns])

    # If we still have no site_id for a row, we can’t backfill it
    existing = set(zip(out["site_id"], out["Species"], out["Scientific name"]))

    new_rows = []
    for sid, grp in df_unique_species.groupby("site_id", dropna=False):
        if pd.isna(sid):
            continue
        base_rows = out[out["site_id"] == sid]
        if base_rows.empty:
            continue

        base = base_rows.iloc[0].to_dict()
        for _, r in grp.iterrows():
            key = (sid, r["Species"], r["Scientific name"])
            if key in existing:
                continue

            new_row = {col: base.get(col, np.nan) for col in out.columns}
            new_row["site_id"] = sid
            new_row["Species"] = r["Species"]
            new_row["Scientific name"] = r["Scientific name"]
            new_row["Plants/ha"] = 0
            if "Per-tree shading (%)" in out.columns:
                new_row["Per-tree shading (%)"] = r["Per-tree shading (%)"]
            new_rows.append(new_row)

    if new_rows:
        out = pd.concat([out, pd.DataFrame(new_rows)], ignore_index=True)

    if added_temp_site:
        out = out.drop(columns=["site_id"])

    sort_cols = [c for c in ["Plot ID","site_id","Species","Scientific name"] if c in out.columns]
    return out.sort_values(sort_cols).reset_index(drop=True)


# Keep 'present' with full columns (including site_id) so we can map site_id by Plot ID.
present_full = canopy_crop_zelie_dict_adjusted["present"]

for name, df in canopy_crop_zelie_dict_adjusted.items():
    canopy_crop_zelie_dict_adjusted[name] = backfill_species_any_sheet(
        sheet_df=df,
        df_unique_species=df_unique_species,
        present_df_for_site_map=present_full
    )
    # Print the adjusted sheets
    print(f"Adjusted Zélie DataFrame for sheet '{name}':\n{tabulate(canopy_crop_zelie_dict_adjusted[name], headers='keys', tablefmt='psql')}")
    print(f"Adjusted Zélie DataFrame for sheet '{name}' has {len(canopy_crop_zelie_dict_adjusted[name])} rows.")



Adjusted Zélie DataFrame for sheet 'present':
+-----+--------------------+----------+------------------+--------------------+-----------------------+-------------+---------------------+------------------------+------------+-------------+-----------------+-----------+-----------+--------------+
|     | Region             | System   |   Plot size (ha) | Species            | Scientific name       |   Plants/ha |   Yield (t/ha/year) |   Per-tree shading (%) |   Latitude |   Longitude |   Elevation (m) |   Plot ID |   site_id | Region_key   |
|-----+--------------------+----------+------------------+--------------------+-----------------------+-------------+---------------------+------------------------+------------+-------------+-----------------+-----------+-----------+--------------|
|   0 | GTM - Alta Verapaz | Cacao    |                1 | Avocado            | Persea americana      |          12 |               nan   |                   50   |    15.5566 |    -90.0506 |             716

In [15]:
for sheet in canopy_crop_zelie_dict_adjusted:
    print(f"Sheet: {sheet}")
    print(tabulate(canopy_crop_zelie_dict_adjusted[sheet], headers='keys', tablefmt='psql'))

Sheet: present
+-----+--------------------+----------+------------------+--------------------+-----------------------+-------------+---------------------+------------------------+------------+-------------+-----------------+-----------+-----------+--------------+
|     | Region             | System   |   Plot size (ha) | Species            | Scientific name       |   Plants/ha |   Yield (t/ha/year) |   Per-tree shading (%) |   Latitude |   Longitude |   Elevation (m) |   Plot ID |   site_id | Region_key   |
|-----+--------------------+----------+------------------+--------------------+-----------------------+-------------+---------------------+------------------------+------------+-------------+-----------------+-----------+-----------+--------------|
|   0 | GTM - Alta Verapaz | Cacao    |                1 | Avocado            | Persea americana      |          12 |               nan   |                   50   |    15.5566 |    -90.0506 |             716 |         0 |         0 | GTM 

#### Role

In [16]:
import pandas as pd

# 1) Define main crops (everything else defaults to "Secondary")
main_species = ["Coffea arabica", "Theobroma cacao"]

# 2) Collect all unique species across the Zélie dict
all_species = set()
for df in canopy_crop_zelie_dict_adjusted.values():
    all_species.update(df["Scientific name"].unique())

# 3) Build role mapping dynamically
role_records = []
for sp in sorted(all_species):
    role = "Main" if sp in main_species else "Secondary"
    role_records.append({"Scientific name": sp, "Role": role})

role_df = pd.DataFrame(role_records)

print("Role DataFrame:")
print(tabulate(role_df, headers="keys", tablefmt="psql"))

# 4) Merge role info into each sheet
for sheet, df in canopy_crop_zelie_dict_adjusted.items():
    canopy_crop_zelie_dict_adjusted[sheet] = df.merge(role_df, on="Scientific name", how="left")


Role DataFrame:
+----+-----------------------+-----------+
|    | Scientific name       | Role      |
|----+-----------------------+-----------|
|  0 | Cedrela odorata       | Secondary |
|  1 | Chamaedorea spp.      | Secondary |
|  2 | Citrus sinensis       | Secondary |
|  3 | Coffea arabica        | Main      |
|  4 | Cordia alliodora      | Secondary |
|  5 | Gliricidia sepium     | Secondary |
|  6 | Inga sapindoides      | Secondary |
|  7 | Inga spp.             | Secondary |
|  8 | Juglans spp.          | Secondary |
|  9 | Mangifera indica      | Secondary |
| 10 | Musa paradisiaca      | Secondary |
| 11 | Musa spp.             | Secondary |
| 12 | Persea americana      | Secondary |
| 13 | Pouteria sapota       | Secondary |
| 14 | Protium copal         | Secondary |
| 15 | Quercus spp.          | Secondary |
| 16 | Spondias mombin       | Secondary |
| 17 | Swietenia macrophylla | Secondary |
| 18 | Theobroma cacao       | Main      |
+----+-----------------------+--------

# Update the present composition to match the input for the cost-benefit

In [17]:
zelie_present_df = canopy_crop_zelie_dict_adjusted['present']

Drop the yield column and add the region key

In [18]:
# Drop the yield
zelie_present_df.drop(columns=["Yield (t/ha/year)"], inplace=True, errors='ignore')
#zelie_present_df["Region_key"] = zelie_present_df["Region"].str.split(" - ", n=1).str[0].str.strip()
print(f"Zélies adjusted files after dropping yield: \n{tabulate(zelie_present_df, headers='keys', tablefmt='psql')}")

Zélies adjusted files after dropping yield: 
+-----+--------------------+----------+------------------+--------------------+-----------------------+-------------+------------------------+------------+-------------+-----------------+-----------+-----------+--------------+-----------+
|     | Region             | System   |   Plot size (ha) | Species            | Scientific name       |   Plants/ha |   Per-tree shading (%) |   Latitude |   Longitude |   Elevation (m) |   Plot ID |   site_id | Region_key   | Role      |
|-----+--------------------+----------+------------------+--------------------+-----------------------+-------------+------------------------+------------+-------------+-----------------+-----------+-----------+--------------+-----------|
|   0 | GTM - Alta Verapaz | Cacao    |                1 | Avocado            | Persea americana      |          12 |                   50   |    15.5566 |    -90.0506 |             716 |         0 |         0 | GTM          | Secondary |

###  Yield and price

In [19]:
import pandas as pd
from tabulate import tabulate

FX = 50.0  # RD$ -> USD, Price/tonne = rd_per_kg * 1000 / FX

# 1) DR fruit table (kg/plant + RD$/kg) -> Tonnes/plant + Price/tonnes (USD)
rows_rd = [
    {"Region_key": "DR", "Scientific name": "Citrus spp.",      "Kg/plant": 40, "rd_per_kg": 40},
    {"Region_key": "DR", "Scientific name": "Persea americana", "Kg/plant": 50, "rd_per_kg": 30},
    {"Region_key": "DR", "Scientific name": "Pouteria sapota",  "Kg/plant": 40, "rd_per_kg": 90},
    {"Region_key": "DR", "Scientific name": "Castanea spp.",    "Kg/plant": 70, "rd_per_kg": 80},
]
df1 = pd.DataFrame(rows_rd)
df1["Tonnes/plant"] = df1["Kg/plant"] / 1000.0
df1["Price/tonnes (USD)"] = (df1["rd_per_kg"] * 1000.0 / FX).round(2)
df1 = df1.drop(columns=["rd_per_kg"])[["Region_key","Scientific name","Kg/plant","Tonnes/plant","Price/tonnes (USD)"]]


# Update your 'typ' list (Coffee already has 1800)
typ = [
    {"Region_key":"DR","Scientific name":"Coffea arabica","Yield (t/ha/yr)":DR_TYPICAL_YIELD["Coffee"],"Plants/ha":DR_TYPICAL_PLANTS_PER_HA["Coffee"],"Price/tonnes (USD)":DR_TYPICAL_PRICE_TONNE_USD["Coffee"]},
    {"Region_key":"DR","Scientific name":"Theobroma cacao","Yield (t/ha/yr)":DR_TYPICAL_YIELD["Cacao"],"Plants/ha":DR_TYPICAL_PLANTS_PER_HA["Cacao"],"Price/tonnes (USD)":DR_TYPICAL_PRICE_TONNE_USD["Cacao"]},
    {"Region_key":"DR","Scientific name":"Musa spp.","Yield (t/ha/yr)": DR_TYPICAL_YIELD["Banana"],"Plants/ha":DR_TYPICAL_PLANTS_PER_HA["Banana"],"Price/tonnes (USD)":DR_TYPICAL_PRICE_TONNE_USD["Banana"]},
]
df2 = pd.DataFrame(typ)
df2["Tonnes/plant"] = (df2["Yield (t/ha/yr)"] / df2["Plants/ha"]).round(6)
df2["Kg/plant"] = (df2["Tonnes/plant"] * 1000).round(3)
df2 = df2[["Region_key","Scientific name","Kg/plant","Tonnes/plant","Price/tonnes (USD)"]]

# 3) Combine
yield_price_df = pd.concat([df1, df2], ignore_index=True)

# 4) Duplicate Citrus genus values to species-level for Zélie’s rows
species_cost_mapping = {
    "Citrus aurantium": "Citrus spp.",
    "Citrus sinensis":  "Citrus spp.",
}
# 3) Duplicate rows for mapped species
def duplicate_species(df, species_cost_mapping):
    """
    Duplicate rows in costs_df_expanded for species in species_cost_mapping.
    Each original species will have its mapped name replaced with the original name.
    """
    expanded_df = df.copy()
    for original_name, mapped_name in species_cost_mapping.items():
        if mapped_name in expanded_df["Scientific name"].values:
            row_to_copy = expanded_df[expanded_df["Scientific name"] == mapped_name].copy()
            row_to_copy["Scientific name"] = original_name
            expanded_df = pd.concat([expanded_df, row_to_copy], ignore_index=True)
    return expanded_df

yield_price_df = duplicate_species(yield_price_df, species_cost_mapping)

# (Optional) drop the genus Citrus spp. row if you only want species-level:
# yield_price_df = yield_price_df[yield_price_df["Scientific name"] != "Citrus spp."]

# Drop the 'Region_key' column if not needed
yield_price_df = yield_price_df.drop(columns=["Region_key"], errors='ignore')

yield_price_df = yield_price_df.sort_values(["Scientific name"]).reset_index(drop=True)
print(tabulate(yield_price_df, headers="keys", tablefmt="psql"))


+----+-------------------+------------+----------------+----------------------+
|    | Scientific name   |   Kg/plant |   Tonnes/plant |   Price/tonnes (USD) |
|----+-------------------+------------+----------------+----------------------|
|  0 | Castanea spp.     |     70     |       0.07     |                 1600 |
|  1 | Citrus aurantium  |     40     |       0.04     |                  800 |
|  2 | Citrus sinensis   |     40     |       0.04     |                  800 |
|  3 | Citrus spp.       |     40     |       0.04     |                  800 |
|  4 | Coffea arabica    |      0.243 |       0.000243 |                 2580 |
|  5 | Musa spp.         |     11.32  |       0.01132  |                  330 |
|  6 | Persea americana  |     50     |       0.05     |                  600 |
|  7 | Pouteria sapota   |     40     |       0.04     |                 1800 |
|  8 | Theobroma cacao   |      0.5   |       0.0005   |                 3760 |
+----+-------------------+------------+-

#### Costs

In [20]:
from heapq import merge
from math import cos
import pandas as pd

# Manual USD costs per tree (RD$ nursery + planting; maintenance = avg of yearly maint.)
costs_dict_usd = {
    "Inga spp.": {
        "Region_key": "DR",
        "Planting cost (per tree)": 1.50,  # (50 + 25) / 50
        "Maintenance cost (per tree)": 0.88,  # (20+30+50+75)/4 / 50
    },
    "Gliricidia sepium": {
        "Region_key": "DR",
        "Planting cost (per tree)": 1.30,  # (30 + 35) / 50
        "Maintenance cost (per tree)": 0.75,  # (20+30+50+50)/4 / 50
    },
    "Citrus spp.": {
        "Region_key": "DR",
        "Planting cost (per tree)": 1.10,  # (30 + 25) / 50
        "Maintenance cost (per tree)": 0.55,  # (20+30+30+30)/4 / 50
    },
    "Persea americana": {
        "Region_key": "DR",
        "Planting cost (per tree)": 1.10,  # (30 + 25) / 50
        "Maintenance cost (per tree)": 0.55,  # same profile
    },
    "Pouteria sapota": {
        "Region_key": "DR",
        "Planting cost (per tree)": 1.50,  # (50 + 25) / 50
        "Maintenance cost (per tree)": 0.55,  # same profile
    },
    "Castanea spp.": {
        "Region_key": "DR",
        "Planting cost (per tree)": 6.80,  # (315 + 25) / 50
        "Maintenance cost (per tree)": 0.55,  # (20+30+30+30)/4 / 50
    },
}

# -> DataFrame
costs_df = (
    pd.DataFrame.from_dict(costs_dict_usd, orient="index")
      .reset_index()
      .rename(columns={"index": "Scientific name"})
)[["Region_key", "Scientific name", "Planting cost (per tree)", "Maintenance cost (per tree)"]]

# 3) Duplicate rows for mapped species
def duplicate_species(df, species_cost_mapping):
    """
    Duplicate rows in costs_df_expanded for species in species_cost_mapping.
    Each original species will have its mapped name replaced with the original name.
    """
    expanded_df = df.copy()
    for original_name, mapped_name in species_cost_mapping.items():
        if mapped_name in expanded_df["Scientific name"].values:
            row_to_copy = expanded_df[expanded_df["Scientific name"] == mapped_name].copy()
            row_to_copy["Scientific name"] = original_name
            expanded_df = pd.concat([expanded_df, row_to_copy], ignore_index=True)
    return expanded_df

# Drop the 'Region_key' column if not needed
costs_df = costs_df.drop(columns=["Region_key"], errors='ignore')

print("Expanded costs DataFrame:")
costs_df = duplicate_species(costs_df, species_cost_mapping)
print(tabulate(costs_df, headers='keys', tablefmt='psql'))



Expanded costs DataFrame:
+----+-------------------+----------------------------+-------------------------------+
|    | Scientific name   |   Planting cost (per tree) |   Maintenance cost (per tree) |
|----+-------------------+----------------------------+-------------------------------|
|  0 | Inga spp.         |                        1.5 |                          0.88 |
|  1 | Gliricidia sepium |                        1.3 |                          0.75 |
|  2 | Citrus spp.       |                        1.1 |                          0.55 |
|  3 | Persea americana  |                        1.1 |                          0.55 |
|  4 | Pouteria sapota   |                        1.5 |                          0.55 |
|  5 | Castanea spp.     |                        6.8 |                          0.55 |
|  6 | Citrus aurantium  |                        1.1 |                          0.55 |
|  7 | Citrus sinensis   |                        1.1 |                          0.55 |
+----+

#### Add the data frame to the Zélies data frame

Merge the data frame and calculate the new yield

In [21]:
# Copy the Zélie present DataFrame to adjust it
zelie_present_adjusted_df = zelie_present_df.copy()

# Add the region key to the DataFrame
#zelie_present_adjusted_df["Region_key"] = zelie_present_adjusted_df["Region"].str.split(" - ", n=1).str[0].str.strip()

# Drop the yield
zelie_present_df.drop(columns=["Yield (t/ha/year)"], inplace=True, errors='ignore')

# Merge yield_price_df with zelie_present_adjusted_df
zelie_present_adjusted_df = zelie_present_adjusted_df.merge(
    yield_price_df,
    on=[ "Scientific name"],
    how="left",
    suffixes=("", "_yield_price")
)

# Calculate the new yield
zelie_present_adjusted_df["Yield (t/ha/year)"] = (
    zelie_present_adjusted_df["Tonnes/plant"] * zelie_present_adjusted_df["Plants/ha"]
)

# Add the costs_df_expanded to zelie_present_adjusted_df
zelie_present_adjusted_df = zelie_present_adjusted_df.merge(costs_df, on=[ "Scientific name"], how="left")

# Reorder columns to match the desired output
new_order_columns = ["Plot ID",
                    #"site_id",
    "Region",
   #"Region_key",
    "Latitude",
    "Longitude",
    "System",
    "Plot size (ha)",
    "Species",
    "Scientific name",
    "Role",
    "Plants/ha",
    "Kg/plant",
    "Tonnes/plant",
    "Yield (t/ha/year)",
    "Price/tonnes (USD)",
    "Per-tree shading (%)",
    "Planting cost (per tree)",
    "Maintenance cost (per tree)"
]
zelie_present_adjusted_df = zelie_present_adjusted_df[new_order_columns]

# Print the final adjusted DataFrame
print(tabulate(zelie_present_adjusted_df, headers='keys', tablefmt='psql'))



+-----+-----------+--------------------+------------+-------------+----------+------------------+--------------------+-----------------------+-----------+-------------+------------+----------------+---------------------+----------------------+------------------------+----------------------------+-------------------------------+
|     |   Plot ID | Region             |   Latitude |   Longitude | System   |   Plot size (ha) | Species            | Scientific name       | Role      |   Plants/ha |   Kg/plant |   Tonnes/plant |   Yield (t/ha/year) |   Price/tonnes (USD) |   Per-tree shading (%) |   Planting cost (per tree) |   Maintenance cost (per tree) |
|-----+-----------+--------------------+------------+-------------+----------+------------------+--------------------+-----------------------+-----------+-------------+------------+----------------+---------------------+----------------------+------------------------+----------------------------+-------------------------------|
|   0 |   

In [22]:
# Columns to check for missing values
check_cols = [
    "Planting cost (per tree)",
    "Maintenance cost (per tree)",
    "Tonnes/plant",
    "Price/tonnes (USD)"
]

# Filter for rows with NaN in any of the check columns
df = zelie_present_adjusted_df.copy()
missing_df = df[df[check_cols].isna().any(axis=1)]

# Keep only Scientific name and Region, drop duplicates
unique_missing = missing_df[["Scientific name", "Yield (t/ha/year)"] + check_cols].drop_duplicates()
print(tabulate(unique_missing, headers='keys', tablefmt='psql'))


+----+-----------------------+---------------------+----------------------------+-------------------------------+----------------+----------------------+
|    | Scientific name       |   Yield (t/ha/year) |   Planting cost (per tree) |   Maintenance cost (per tree) |   Tonnes/plant |   Price/tonnes (USD) |
|----+-----------------------+---------------------+----------------------------+-------------------------------+----------------+----------------------|
|  1 | Musa paradisiaca      |               nan   |                      nan   |                        nan    |     nan        |                  nan |
|  2 | Musa spp.             |                 0   |                      nan   |                        nan    |       0.01132  |                  330 |
|  3 | Theobroma cacao       |                 0.5 |                      nan   |                        nan    |       0.0005   |                 3760 |
|  4 | Cedrela odorata       |               nan   |                      na

Calculate the new  Yield (t/ha/year)

In [23]:
import pandas as pd
import numpy as np

# df is your merged Zélie table (the one you showed last)
df = zelie_present_adjusted_df.copy()

# Make sure inputs are numeric
df["Plants/ha"] = pd.to_numeric(df["Plants/ha"], errors="coerce")
df["Tonnes/plant"] = pd.to_numeric(df["Tonnes/plant"], errors="coerce")
df["Yield (t/ha/year)"] = pd.to_numeric(df["Yield (t/ha/year)"], errors="coerce")

# Candidate yield per ha
candidate = df["Plants/ha"] * df["Tonnes/plant"]

# Fill only where Yield (t/ha/year) is NaN
mask = df["Yield (t/ha/year)"].isna() & candidate.notna()
df.loc[mask, "Yield (t/ha/year)"] = candidate[mask]

# (optional) round to 2–3 decimals
df["Yield (t/ha/year)"] = df["Yield (t/ha/year)"].round(3)

# Update the DataFrame with the new Yield (t/ha/year)
zelie_present_adjusted_df = df

print("Updated DataFrame with Yield (t/ha/year):")
print(tabulate(zelie_present_adjusted_df, headers='keys', tablefmt='psql'))


Updated DataFrame with Yield (t/ha/year):
+-----+-----------+--------------------+------------+-------------+----------+------------------+--------------------+-----------------------+-----------+-------------+------------+----------------+---------------------+----------------------+------------------------+----------------------------+-------------------------------+
|     |   Plot ID | Region             |   Latitude |   Longitude | System   |   Plot size (ha) | Species            | Scientific name       | Role      |   Plants/ha |   Kg/plant |   Tonnes/plant |   Yield (t/ha/year) |   Price/tonnes (USD) |   Per-tree shading (%) |   Planting cost (per tree) |   Maintenance cost (per tree) |
|-----+-----------+--------------------+------------+-------------+----------+------------------+--------------------+-----------------------+-----------+-------------+------------+----------------+---------------------+----------------------+------------------------+----------------------------+-

# Adjust prices to make resonable analysis

In [24]:
import pandas as pd

def adjust_secondary_econ(
    df: pd.DataFrame,
    price_mult: float = 1.0,       # e.g., 0.8 lowers prices by 20%
    plant_cost_mult: float = 1.0,  # e.g., 0.9 lowers planting cost by 10%
    maint_cost_mult: float = 1.0,  # e.g., 0.75 lowers maintenance by 25%
) -> pd.DataFrame:
    """
    Return a copy of df where rows with Role == 'Secondary' have their
    price and per-tree costs scaled by the given multipliers.
    (NaNs remain NaN.)
    """
    out = df.copy()
    sec = out["Role"].eq("Secondary")

    if "Price/tonnes (USD)" in out:
        out.loc[sec, "Price/tonnes (USD)"] = out.loc[sec, "Price/tonnes (USD)"] * price_mult
    if "Planting cost (per tree)" in out:
        out.loc[sec, "Planting cost (per tree)"] = out.loc[sec, "Planting cost (per tree)"] * plant_cost_mult
    if "Maintenance cost (per tree)" in out:
        out.loc[sec, "Maintenance cost (per tree)"] = out.loc[sec, "Maintenance cost (per tree)"] * maint_cost_mult

    return out


# 1) Make fruit shade less dominant and bump O&M for all secondary trees
zelie_present_adjusted_incl_price_df = adjust_secondary_econ(
    zelie_present_adjusted_df,
    price_mult=0.1,                      # 70% of previous price for Secondary (fruit shade)
    plant_cost_mult=1.0,                 # 0% chnage planting cost
    maint_cost_mult=1.0,                 # 0% chnage maintenance
)

# Generate alterantives

- Change crops, e.g., coffe -> cacao
- Increase canopy (%) from present
- Add mixture for given condition, e.g., heat resilient (already taken care of in Zélies code)


In [25]:
# import pandas as pd
# from tabulate import tabulate

excel_dict = copy.deepcopy(canopy_crop_zelie_dict_adjusted)

# Update the 'present' sheet with the adjusted DataFrame
excel_dict["present"] = zelie_present_adjusted_incl_price_df

# # Define the range of adjustments for canopy (e.g., -10% to +10% in steps of 5%)
# adjustments = np.linspace(-0.80, 0.80, num=10)  # from -10% to +10% in 5 steps

# results = {}  # store DataFrames keyed by adjustment value

# for adj in adjustments:
#     df_adjusted = zelie_present_adjusted_df[store_columns + ["Role"]].copy()

#     # Apply adjustment only to rows where Role == "Secondary"
#     df_adjusted["Plants/ha"] = df_adjusted["Plants/ha"].where(
#         df_adjusted["Role"] != "Secondary",
#         df_adjusted["Plants/ha"] * (1 + adj)
#     )

#     # Round to nearest integer
#     df_adjusted["Plants/ha"] = df_adjusted["Plants/ha"].round().astype(int)

#     # Store the adjusted DataFrame
#     results[adj] = df_adjusted

#     # Optional: print a sample for inspection
#     print(f"\nAdjustment: {adj*100:.0f}%")
#     print(tabulate(df_adjusted, headers="keys", tablefmt="psql"))

#     # Save the adjusted DataFrame to the dictionary
#     excel_dict[f"present_adjusted_{int(adj*100)} %"] = df_adjusted



# Switch cacaco to coffe but have the same present canopy compostion

In [26]:
# df_new_cacao = canopy_crop_zelie_dict_adjusted['present'][store_columns + ["Role"]].copy()
# print(tabulate(df_new_cacao, headers='keys', tablefmt='psql'))

# # Update the Scientific name for Cacao by setting Plants/ha to 0
# df_new_cacao.loc[df_new_cacao['Scientific name'] == 'Theobroma cacao', 'Plants/ha'] = 0
# # Update the Scientific name for Coffe by setting 
# df_new_cacao.loc[df_new_cacao['Scientific name'] == 'Coffea arabica', 'Plants/ha'] = DR_TYPICAL_PLANTS_PER_HA["Coffee"]

# print(tabulate(df_new_cacao, headers='keys', tablefmt='psql'))

# # put back to the excel_dict
# excel_dict['cacao_to_coffee'] = df_new_cacao




## Save excel version 

In [27]:
# Save the adjusted DataFrame to an Excel excel_file
file_name = file_name_Zelie.replace(".xlsx", "_adjusted_canopy_crop_composition.xlsx")
output_file = OUTPUT_DIR / file_name
with pd.ExcelWriter(output_file, engine='xlsxwriter') as writer:
    for sheet_name, df in excel_dict.items():
        df.to_excel(writer, sheet_name=sheet_name, index=False)
        print(f"Saved sheet: {sheet_name} with {len(df)} rows.")


Saved sheet: present with 190 rows.
Saved sheet: shade_-50 with 190 rows.
Saved sheet: shade_-30 with 190 rows.
Saved sheet: shade_-25 with 190 rows.
Saved sheet: shade_-20 with 190 rows.
Saved sheet: shade_-15 with 190 rows.
Saved sheet: shade_-10 with 190 rows.
Saved sheet: shade_-5 with 190 rows.
Saved sheet: shade_+5 with 190 rows.
Saved sheet: shade_+10 with 190 rows.
Saved sheet: shade_+15 with 190 rows.
Saved sheet: shade_+20 with 190 rows.
Saved sheet: shade_+25 with 190 rows.
Saved sheet: shade_+30 with 190 rows.
Saved sheet: shade_+50 with 190 rows.
Saved sheet: cacao_to_coffee with 190 rows.
